In [1]:
import csv
import sys
import re
import matplotlib.pyplot as plt
import numpy as np

# Cutoff for rare variants 

In [2]:
cutoff  = 0.00001

In [3]:
def writelist (mylist, myfile): 
    fp =  open(myfile,'a+')
    for idx,item in enumerate(mylist):
            fp.write(item)
            fp.write('\t')
    fp.write('\n')
    fp.close()

In [4]:
def intersections(list1, list2):
    intersection_list = []
    for i in list1: 
        if i in list2: 
                intersection_list.append(i)
    return intersection_list

In [5]:
def difference(list1, list2):
    unique_list = []
    for i in list1: 
        if i not in list2: 
                unique_list.append(i)
    return unique_list

In [6]:
def exist (list1, list2): 
    found_list = []
    for i in list1: 
            if i not in found_list: 
                    found_list.append(i)
    for i in list2: 
            if i not in found_list: 
                    found_list.append(i)
    return found_list 

In [7]:
def common_variants(variants1, variants2, outfile): 
        fp = open(outfile, 'w')
        for i in variants1: 
               for j in variants2: 
                     count = 0 
                     for a in range(0,4): 
                            if i[a] == j[a]:
                                count +=1
                     if count == 4: 
                          for item in (i):
                                fp.write(item)
                                fp.write('\t')
                          fp.write('\n')

# Read annovar annotation file

In [13]:
 def load_annovar(filename):
    rows = []
    exacall = []
    gnomall = []
    rs = []
    gene = []
    pathogenics = []
    functions = []
    exonicfunctions =[]
    fp = open(filename, 'r')
    r = csv.reader(fp, delimiter='\t')
    for n, row in enumerate(r):
        if n == 0: 
                for i in range(0,len(row) ): 
                     print (i,row[i])
                continue
        exonicfunctions.append(row[5])
        exacall.append(row[11])
        gnomall.append(row[119])
        rs.append(row[19])
        rows.append(row)
        gene.append(row[6])
        functions.append(row[8])
        pathogenics.append(row[54])
        #if row[6] == 'GBA':
         #       print('genes, Exac and Gnom are:', row[6], row[11], row[119])
    return rows, functions, exonicfunctions, rs, gene,  exacall , gnomall, pathogenics

# Filter for rare MAF

In [9]:
def maf_filter(mfile): 
    rows, functions,exfunc, rs,gene, exacall, gnomall, pathogenics = load_annovar(mfile)
    count = 0
    outfile = mfile.split('.')[0] + '_raremaf.txt'
    raremaf_genes = []
    variants_list = [] 
    for i in range(0,len(exacall) ): 
        if exfunc[i] =='exonic':
                    #========================================
                    #If both gnomAD and ExAC is '.'
                    #========================================
                    if (gnomall[i] != '.') and (exacall[i] !='.'):
                        if   (float(gnomall[i]) < cutoff) and (float(exacall[i]) < cutoff) : 
                            count += 1 
                            writelist(rows[i], outfile)
                            variants_list.append(rows[i])
                            print(rs[i], exfunc[i],gene[i], exacall[i],gnomall[i])
                            if gene[i] not in raremaf_genes: 
                                    raremaf_genes.append(gene[i])
                    #========================================
                    #If both gnomAD is '.' and ExAC is not '.'
                    #========================================               
                    elif (gnomall[i] == '.') and (exacall[i] != '.') and (float(exacall[i]) < cutoff) :
                            count += 1 
                            writelist(rows[i], outfile)                    
                            variants_list.append(rows[i])
                            print(rs[i], exfunc[i],gene[i], exacall[i],gnomall[i])
                            if gene[i] not in raremaf_genes: 
                                    raremaf_genes.append(gene[i])
                    #========================================
                    #If both ExAC is '.' and gnomAD is not '.'
                    #======================================== 
                    elif (exacall[i] =='.') and (gnomall[i] != '.') and (float (gnomall[i]) < cutoff)  :
                            count += 1 
                            writelist(rows[i], outfile)
                            variants_list.append(rows[i])
                            print(rs[i], exfunc[i],gene[i], exacall[i],gnomall[i])
                            if gene[i] not in  raremaf_genes: 
                                     raremaf_genes.append(gene[i])
    return(raremaf_genes,  variants_list)

# Filter for pathogenic variants

In [10]:
def pathogenic_filter(mfile):
    rows, functions,exfunc, rs,gene, exacall, gnomall, pathogenics = load_annovar(mfile)
    count = 0
    pathogenic_genes = []    
    variants_list = [] 
    outfile = mfile.split('.')[0]+'pathogenic.txt'
    for i in range(0,len(exacall) ): 
        if 'athogenic' in pathogenics[i]: 
            if (exfunc[i] =='exonic'): 
                if ((exacall[i] == '.') or( float(exacall[i]) < cutoff)) and  ((gnomall[i] =='.') or (float(gnomall[i]) < cutoff)):
                        count += 1 
                        writelist(rows[i], outfile)
                        variants_list.append(rows[i])
                        print(rs[i], gene[i], exfunc[i],exacall[i],gnomall[i], pathogenics[i])
                        if gene[i] not in pathogenic_genes:
                                pathogenic_genes.append(gene[i])
    print ('Number of variants that are rare and pathogenic cutoff <', cutoff, 'is:',count)
    return pathogenic_genes, variants_list 

# Filter for no MAF variants

In [11]:
def nomaf_filter(mfile):    
    rows, functions,exfunc, rs,gene, exacall, gnomall, pathogenics = load_annovar(mfile)
    count = 0
    nomafs_genes = []
    nomafvariants = [] 
    outfile = mfile.split('.')[0]+'nomaf.txt'
    
    for i in range(0,len(exacall) ): 
        if (exacall[i] == '.') and (gnomall[i] =='.') and exfunc[i] =='exonic':
                count += 1 
                writelist(rows[i], outfile)
                if gene[i] not in nomafs_genes:
                        nomafs_genes.append(gene[i])
                var = rs[i]+'.'+exfunc[i]+'.'+gene[i]
                if var not in  nomafvariants: 
                         nomafvariants.append(var)
    print ('Number of variants with no MAf in both exacall and gnomAD <', cutoff, 'is:',count)
    return nomafs_genes,  nomafvariants 

# F20 Analysis

In [ ]:
f20m1_raremaf, f20m1_variants  = maf_filter('S0959.hg19_multianno.txt')
print('Done')

0 Chr
1 Start
2 End
3 Ref
4 Alt
5 Func.refGene
6 Gene.refGene
7 GeneDetail.refGene
8 ExonicFunc.refGene
9 AAChange.refGene
10 Func.ensGene
11 Gene.ensGene
12 GeneDetail.ensGene
13 ExonicFunc.ensGene
14 AAChange.ensGene
15 cytoBand
16 ExAC_ALL
17 ExAC_AFR
18 ExAC_AMR
19 ExAC_EAS
20 ExAC_FIN
21 ExAC_NFE
22 ExAC_OTH
23 ExAC_SAS
24 avsnp147
25 SIFT_score
26 SIFT_pred
27 Polyphen2_HDIV_score
28 Polyphen2_HDIV_pred
29 Polyphen2_HVAR_score
30 Polyphen2_HVAR_pred
31 LRT_score
32 LRT_pred
33 MutationTaster_score
34 MutationTaster_pred
35 MutationAssessor_score
36 MutationAssessor_pred
37 FATHMM_score
38 FATHMM_pred
39 PROVEAN_score
40 PROVEAN_pred
41 VEST3_score
42 CADD_raw
43 CADD_phred
44 DANN_score
45 fathmm-MKL_coding_score
46 fathmm-MKL_coding_pred
47 MetaSVM_score
48 MetaSVM_pred
49 MetaLR_score
50 MetaLR_pred
51 integrated_fitCons_score
52 integrated_confidence_value
53 GERP++_RS
54 phyloP7way_vertebrate
55 phyloP20way_mammalian
56 phastCons7way_vertebrate
57 phastCons20way_mammalian
58 

In [ ]:
f20m2_raremaf , f20m2_variants= maf_filter('S0960.hg19_multianno.txt')

In [ ]:
f20m7_raremaf , f20m7_variants= maf_filter('S0961.hg19_multianno.txt')

In [ ]:
f20m1_pathogenic,f20m1_pathogenic_variants   = pathogenic_filter('S0959.hg19_multianno.txt')

In [ ]:
f20m2_pathogenic,f20m2_pathogenic_variants = pathogenic_filter('S0960.hg19_multianno.txt')

In [ ]:
f20m7_pathogenic,f20m7_pathogenic_variants = pathogenic_filter('S0961.hg19_multianno.txt')

In [ ]:
f20m1_nomaf = nomaf_filter('S0959.hg19_multianno.txt')

In [ ]:
f20m2_nomaf = nomaf_filter('S0960.hg19_multianno.txt')

In [ ]:
f20m7_nomaf = nomaf_filter('S0961.hg19_multianno.txt')

In [ ]:
common_variants(f366m1_variants,f366m2_variants, 'f366m1m2.rare.common.txt')
common_variants(f366m1_pathogenic_variants,f366m2_pathogenic_variants, 'f366m1m2.pathogenic.common.txt')

In [ ]:
f366m1_m2_raremaf = intersections(f366m2_raremaf,f366m1_raremaf)
print(f366m1_m2_raremaf)

# F20 pathogenic variants common between M1 and M2

In [ ]:
f366m1_m2_pathogenic = intersections(f366m1_pathogenic , f366m2_pathogenic )
print(f366m1_m2_pathogenic )

# F20  no MAFs common between M1 and M2

In [ ]:
f366m1_m2_nomaf   = difference(f366m1_nomaf  , f366m2_nomaf  )
print(f366m1_m2_nomaf )